In [1]:
import numpy as np
import pandas as pd
import copy

import math
import scipy.stats as stats

import datetime
from dateutil.relativedelta import relativedelta

!pip install pypsa
import pypsa
# conda install -c conda-forge cartopy
# import cartopy

import matplotlib.pyplot as plt

# import warnings
# warnings.filterwarnings('ignore')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 kB 14.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.2/63.2 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.0/354.0 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 60.0 MB/s eta 0:00:00
  Created wheel for pypsa: filename=pypsa-0.24.0-py3-none-any.whl size=162917 sha256=21a5f9c0fa65665759358a5726ec8cf11e4e483db48bd51e6e37fef8b95dfa75
  Stored in directory: /root/.cache/pip/wheels/f4/dc/ef/ac237ce4afe4c5543a73b86f96b91e68bcfbba7c71e040081c
  Crea

/usr/local/lib/python3.10/dist-packages/pypsa/networkclustering.py:16: UserWarning:

The namespace `pypsa.networkclustering` is deprecated and will be removed in PyPSA v0.24. Please use `pypsa.clustering.spatial instead`. 



In [2]:
# Import sample network

network_sample = pypsa.examples.ac_dc_meshed(from_master=True)

In [3]:
# Buses in the network

# name, v_nom, x, y, carrier seems to be necessary for buses
# name: Unique name
# v_nom: Nominal voltage
# x: Position (e.g. longitude)
# y: Position (e.g. latitude)
# carrier: Energy carrier (AC or DC for electrical buses, or heat or gas)

# How to add a bus
# network.add('Bus', 'Name', v_nom=...)

network_sample.buses

,v_nom,carrier,x,y,type,unit,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,sub_network
Bus,,,,,,,,,,,
London,380.0,AC,-0.13,51.50,,None,1.0,0.0,inf,PQ,
Norwich,380.0,AC,1.30,52.60,,None,1.0,0.0,inf,PQ,
Norwich DC,200.0,DC,1.30,52.50,,None,1.0,0.0,inf,PQ,
Manchester,380.0,AC,-2.20,53.47,,None,1.0,0.0,inf,PQ,
Bremen,380.0,AC,8.80,53.08,,None,1.0,0.0,inf,PQ,
Bremen DC,200.0,DC,8.80,52.98,,None,1.0,0.0,inf,PQ,
Frankfurt,380.0,AC,8.70,50.12,,None,1.0,0.0,inf,PQ,
Norway,380.0,AC,10.75,60.00,,None,1.0,0.0,inf,PQ,
Norway DC,200.0,DC,10.75,60.00,,None,1.0,0.0,inf,PQ,


In [4]:
# Lines in the network

# name, bus0 and bus1, x or r, s_nom, capital_cost seems to be necessary for lines
# name: Unique name
# bus0 and bus1: Buses to which line is attached
# x: Series reactance, must be non-zero for AC branch in linear power flow
# r: Series resistance, must be non-zero for DC branch in linear power flow
# s_nom: Limit of apparent power which can pass through line
# capital_cost: Capital cost of extending s_nom by 1 MVA

# How to add a line
# network.add('Line', 'Name', bus0=...)

network_sample.lines

,bus0,bus1,capital_cost,s_nom,s_nom_extendable,x,r,type,g,b,...,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt
Line,,,,,,,,,,,,,,,,,,,,,
0,London,Manchester,0.136716,40000.0,True,0.796878,0.000000,,0.0,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Manchester,Norwich,0.133492,40000.0,True,0.391560,0.000000,,0.0,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Bremen DC,Norwich DC,0.008673,40000.0,True,0.000000,0.212604,,0.0,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Norwich DC,Norway DC,0.129126,40000.0,True,0.000000,0.486164,,0.0,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Norway DC,Bremen DC,0.062430,40000.0,True,0.000000,0.428727,,0.0,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Norwich,London,0.021852,40000.0,True,0.238800,0.000000,,0.0,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Bremen,Frankfurt,0.200000,40000.0,True,0.400000,0.000000,,0.0,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# Generators in the network

# name, bus, p_nom, carrier, marginal_cost, marginal_cost, efficiency seems to be necessary for generators
# name: Unique name
# bus: Bus to which generator is attached
# p_nom: Nominal power for limits in optimization
# carrier: Prime mover energy carrier (e.g. coal, gas, wind, solar)
# marginal_cost: Marginal cost of production of 1 MWh
# capital_cost: Capital cost of extending p_nom by 1 MW
# efficiency: Ratio between primary energy and electrical energy

# How to add a generator
# network.add('Generator', 'Name', bus=...)

network_sample.generators

,bus,capital_cost,efficiency,marginal_cost,p_nom,p_nom_extendable,p_nom_min,carrier,control,type,...,shut_down_cost,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,
Manchester Wind,Manchester,2793.651603,1.000000,0.110000,80.0,True,100.0,wind,PQ,,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
Manchester Gas,Manchester,196.615168,0.350026,4.532368,50000.0,True,0.0,gas,PQ,,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
Norway Wind,Norway,2184.374796,1.000000,0.090000,100.0,True,100.0,wind,PQ,,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
Norway Gas,Norway,158.251250,0.356836,5.892845,20000.0,True,0.0,gas,PQ,,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
Frankfurt Wind,Frankfurt,2129.456122,1.000000,0.100000,110.0,True,100.0,wind,PQ,,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
Frankfurt Gas,Frankfurt,102.676953,0.351666,4.086322,80000.0,True,0.0,gas,PQ,,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0


In [6]:
# Loads in the network

# name, bus, carrier, p_set, q_set seems to be necessary for generators
# name: Unique name
# bus: Bus to which load is attached
# carrier: Energy carrier (AC or DC for electrical buses, or heat or gas)
# p_set: Active power consumption
# q_set: Reactive power consumption

# How to add a load
# network.add('Load', 'Name', bus=...)

network_sample.loads

,bus,carrier,type,p_set,q_set,sign
Load,,,,,,
London,London,,,0.0,0.0,-1.0
Frankfurt,Frankfurt,,,0.0,0.0,-1.0
Norway,Norway,,,0.0,0.0,-1.0
Norwich,Norwich,,,0.0,0.0,-1.0
Bremen,Bremen,,,0.0,0.0,-1.0
Manchester,Manchester,,,0.0,0.0,-1.0


In [9]:
# Active power flow on the lines

network_sample.lines_t.p0

,0,1,2,3,4,5,6
snapshot,,,,,,,
2015-01-01 00:00:00,-129.791210,321.467598,0.0,0.0,0.0,-93.994966,0.0
2015-01-01 01:00:00,-503.473900,735.956808,0.0,0.0,0.0,473.350662,0.0
2015-01-01 02:00:00,-225.483574,443.580091,0.0,0.0,0.0,25.103738,0.0
2015-01-01 03:00:00,-209.452290,474.260394,0.0,0.0,0.0,-78.699145,0.0
2015-01-01 04:00:00,-126.587247,242.672780,0.0,0.0,0.0,24.512922,0.0
2015-01-01 05:00:00,-628.844903,1094.988415,0.0,0.0,0.0,303.012149,0.0
2015-01-01 06:00:00,-273.933241,547.785726,0.0,0.0,0.0,15.915046,0.0
2015-01-01 07:00:00,-388.200425,499.656364,0.0,0.0,0.0,476.142897,0.0
2015-01-01 08:00:00,-570.695284,1088.941048,0.0,0.0,0.0,118.881980,0.0


In [10]:
# Voltage angles on the buses

network_sample.buses_t.v_ang * 180 / np.pi

Bus,London,Norwich,Norwich DC,Manchester,Bremen,Bremen DC,Frankfurt,Norway,Norway DC
snapshot,,,,,,,,,
2015-01-01 00:00:00,-0.041039,-0.049945,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015-01-01 01:00:00,-0.159194,-0.114342,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015-01-01 02:00:00,-0.071296,-0.068917,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015-01-01 03:00:00,-0.066227,-0.073684,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015-01-01 04:00:00,-0.040026,-0.037703,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015-01-01 05:00:00,-0.198836,-0.170124,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015-01-01 06:00:00,-0.086615,-0.085107,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015-01-01 07:00:00,-0.122745,-0.077629,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015-01-01 08:00:00,-0.180449,-0.169185,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Mangitudes of voltage angles on the buses

network_sample.buses_t.v_mag_pu

Bus,London,Norwich,Norwich DC,Manchester,Bremen,Bremen DC,Frankfurt,Norway,Norway DC
snapshot,,,,,,,,,
2015-01-01 00:00:00,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2015-01-01 01:00:00,0.999996,0.999997,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2015-01-01 02:00:00,0.999999,0.999999,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2015-01-01 03:00:00,0.999999,0.999999,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2015-01-01 04:00:00,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2015-01-01 05:00:00,0.999995,0.999995,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2015-01-01 06:00:00,0.999999,0.999999,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2015-01-01 07:00:00,0.999998,0.999998,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2015-01-01 08:00:00,0.999995,0.999995,1.0,1.0,1.0,1.0,1.0,1.0,1.0
